# Laboratorio no calificado: Esquema iterativo con TFX y metadatos ML


En este cuaderno, podrá revisar cómo actualizar un esquema inferido y guardar el resultado en el almacén de metadatos utilizado por TFX. Como se mencionó anteriormente, los componentes de TFX obtienen información de esta base de datos antes de realizar las ejecuciones. Por lo tanto, si usted va a curar un esquema, tendrá que guardar esto como un artefacto en el almacén de metadatos. Podrás ver cómo se hace en el siguiente ejercicio.

Después, también practicarás el acceso al almacén de metadatos TFX y verás cómo puedes rastrear el linaje de un artefacto.

In [1]:
import tensorflow as tf
import tensorflow_data_validation as tfdv

from tfx import v1 as tfx

from tfx.types import standard_artifacts

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
from tensorflow_metadata.proto.v0 import schema_pb2

import os
import pprint
pp = pprint.PrettyPrinter()

### Definir rutas

Para familiarizarse, volverá a utilizar el conjunto de datos [Census Income dataset](https://archive.ics.uci.edu/ml/datasets/Adult) de los laboratorios no calificados de las semanas anteriores. Utilizará las mismas rutas de acceso a los datos brutos y a los archivos de canalización que se muestran a continuación.

In [2]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'

# directory of the raw data files
_data_root = './data/census_data'

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'adult.data')

## Canalización de datos

Cada componente TFX que se utiliza acepta y genera artefactos que son instancias de los diferentes tipos de artefactos que TFX ha configurado en el almacén de metadatos. Las propiedades de estas instancias se muestran ordenadamente en una tabla en las salidas de `context.run()`. TFX hace todo esto por ti, así que sólo tienes que inspeccionar la salida de cada componente para saber qué propiedad del artefacto puedes pasar al siguiente componente (por ejemplo, las `salidas['ejemplos']` de `ExampleGen` se pueden pasar a `StatisticsGen`).

Dado que ya has utilizado este conjunto de datos, nos limitaremos a repasar rápidamente `ExampleGen`, `StatisticsGen` y `SchemaGen`. Los nuevos conceptos se discutirán después de dichos componentes.

### Crear el Contexto Interactivo

In [3]:
# Inicializar el Contexto Interactivo.
# Si dejas `_pipeline_root` en blanco, el db se creará en un directorio temporal.
context = InteractiveContext(pipeline_root=_pipeline_root)

### ExampleGen

In [4]:
# Instanciar ExampleGen con el conjunto de datos CSV de entrada
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

# Ejecutar el componente
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "./pipeline/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:3974460,xor_checksum:1635234403,sum_checksum:1635234403"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### StatisticsGen

In [5]:
# Instanciar StatisticsGen con el conjunto de datos ingestado por ExampleGen
statistics_gen = tfx.components.StatisticsGen(
                                            examples = example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### SchemaGen

In [6]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = tfx.components.SchemaGen(
                                    statistics = statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [7]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'education',STRING,required,,'education'
'label',STRING,required,,'label'
'marital-status',STRING,required,,'marital-status'
'native-country',STRING,required,,'native-country'
'occupation',STRING,required,,'occupation'
'race',STRING,required,,'race'
'relationship',STRING,required,,'relationship'
'sex',STRING,required,,'sex'
'workclass',STRING,required,,'workclass'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'label',"' <=50K', ' >50K'"
'marital-status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native-country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', ' Holand-Netherlands'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


### Curar el esquema

Ahora que tiene el esquema inferido, puede proceder a revisarlo para que sea más sólido. Por ejemplo, puedes restringir la edad como hiciste en la semana 1. En primer lugar, tienes que cargar el buffer del protocolo `Schema` desde el almacén de metadatos. Puedes hacerlo obteniendo la uri del esquema de la salida de `SchemaGen` y luego utilizar el método `load_schema_text()` de TFDV.

In [8]:
# Get the schema uri
schema_uri = schema_gen.outputs['schema']._artifacts[0].uri

# Get the schema pbtxt file from the SchemaGen output
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

Con esto, ahora puedes hacer cambios en el esquema como antes. Para el propósito de este ejercicio, sólo modificará el dominio de la edad, pero siéntase libre de añadir más si lo desea.

In [9]:
# Restrict the range of the `age` feature
tfdv.set_domain(schema, 'age', schema_pb2.IntDomain(name='age', min = 17, max = 90))

# Display the modified schema. Notice the `Domain` column of `age`.
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'education',STRING,required,,'education'
'label',STRING,required,,'label'
'marital-status',STRING,required,,'marital-status'
'native-country',STRING,required,,'native-country'
'occupation',STRING,required,,'occupation'
'race',STRING,required,,'race'
'relationship',STRING,required,,'relationship'
'sex',STRING,required,,'sex'
'workclass',STRING,required,,'workclass'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'label',"' <=50K', ' >50K'"
'marital-status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native-country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', ' Holand-Netherlands'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


### Entornos del esquema

Por defecto, su esquema buscará todas las características declaradas anteriormente, incluyendo la etiqueta. Sin embargo, cuando el modelo se sirve para la inferencia, obtendrá conjuntos de datos que no tendrán la etiqueta porque esa es la característica que el modelo estará tratando de predecir. Hay que configurar el pipeline para que no salte una alarma cuando se reciba este tipo de conjunto de datos.

Puede hacerlo con [entornos de esquema](https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic#schema_environments). En primer lugar, tendrá que declarar los entornos de entrenamiento y de servicio, y luego configurar el esquema de servicio para que no vigile la presencia de etiquetas. Vea cómo se implementa a continuación.

In [10]:
# Crear los entornos de los esquemas para la formación y el servicio
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Omitir la etiqueta del entorno de servicio
tfdv.get_feature(schema, 'label').not_in_environment.append('SERVING')

Ahora puede congelar el esquema curado y guardarlo en un directorio local.

In [11]:
# Declarar la ruta del directorio del esquema actualizado
_updated_schema_dir = f'{_pipeline_root}/updated_schema'

# Creamos el directorio
!mkdir -p {_updated_schema_dir}

# Declarar la ruta de acceso al archivo del esquema
schema_file = os.path.join(_updated_schema_dir, 'schema.pbtxt')

# Guardar el esquema curado en dicho archivo
tfdv.write_schema_text(schema, schema_file)

### ImportSchemaGen

Ahora que el esquema ha sido guardado, necesitas crear un artefacto en el almacén de metadatos que apunte a él. TFX proporciona el componente [ImporterSchemaGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/ImportSchemaGen) utilizado para importar un esquema curado a ML Metadata. Simplemente hay que especificar la URL del archivo de esquema revisado.

In [12]:
# Utilice ImportSchemaGen para poner el esquema curado en ML Metadata
user_schema_importer = tfx.components.ImportSchemaGen(schema_file=schema_file)

# Run the component
context.run(user_schema_importer, enable_cache=False)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 4
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 18
        uri: "./pipeline/ImportSchemaGen/schema/4"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ImportSchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Si pasas la salida del componente a `context.show()`, entonces deberías ver el esquema.

In [13]:
# See the result
context.show(user_schema_importer.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'education',STRING,required,,'education'
'label',STRING,required,,'label'
'marital-status',STRING,required,,'marital-status'
'native-country',STRING,required,,'native-country'
'occupation',STRING,required,,'occupation'
'race',STRING,required,,'race'
'relationship',STRING,required,,'relationship'
'sex',STRING,required,,'sex'
'workclass',STRING,required,,'workclass'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'label',"' <=50K', ' >50K'"
'marital-status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native-country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', ' Holand-Netherlands'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


### ExampleValidator

A continuación, puede utilizar este nuevo artefacto como entrada a los otros componentes de la tubería. Vea cómo se utiliza como el argumento `schema` en `ExampleValidator` a continuación.

In [14]:
# Instanciar ExampleValidator con los datos ingeridos de StatisticsGen y SchemaGen
example_validator = tfx.components.ExampleValidator(
                                                   statistics = statistics_gen.outputs['statistics'],
                                                   schema     = user_schema_importer.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 5
        type_id: 21
        uri: "./pipeline/ExampleValidator/anomalies/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

### Práctica con los metadatos de ML

Llegados a este punto, debería dedicar algún tiempo a explorar el contenido del almacén de metadatos guardado por sus ejecuciones de componentes. Esto le permitirá practicar el seguimiento de los artefactos y cómo se relacionan entre sí. Esto implica mirar los artefactos, las ejecuciones y los eventos. Esta habilidad le permitirá recuperar artefactos relacionados incluso sin ver el código de la ejecución de entrenamiento. Todo lo que necesitas es acceder al almacén de metadatos. 

Vea cómo se rastrean los IDs de los artefactos de entrada a una instancia de `ExampleAnomalies` en las siguientes celdas. Si tiene este cuaderno, entonces ya sabrá que utiliza la salida de StatisticsGen para esta ejecución y también el esquema curado que importó. Sin embargo, si ya tiene cientos de ejecuciones de entrenamiento e iteraciones de parámetros, puede que le resulte difícil rastrear cuál es cuál. Ahí es donde el almacén de metadatos será útil. Dado que registra información sobre una ejecución específica de la tubería, usted podrá rastrear las entradas y salidas de un artefacto en particular.

Empezarás por establecer la configuración de la conexión al almacén de metadatos. 

In [16]:
# Import mlmd and utilities
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2

# Obtener la configuración de la conexión para conectarse al almacén de metadatos del contexto
connection_config = context.metadata_connection_config

# Instanciar una instancia de MetadataStore con la configuración de la conexión
store = mlmd.MetadataStore(connection_config)

A continuación, vamos a ver qué tipos de artefactos están disponibles en el almacén de metadatos.

In [17]:
# Get artifact types
artifact_types = store.get_artifact_types()

# Print the results
[artifact_type.name for artifact_type in artifact_types]

['Examples', 'ExampleStatistics', 'Schema', 'ExampleAnomalies']

Si obtienes los artefactos de tipo `Schema`, verás que hay dos entradas. Una es la inferida y la otra es la que has importado. Al final de este ejercicio, puedes verificar que el esquema curado es el que se utiliza para la ejecución del `ExampleValidator` que vamos a investigar.

In [18]:
# Obtener tipos de artefactos
schema_list = store.get_artifacts_by_type('Schema')

[(f'schema uri: {schema.uri}', f'schema id:{schema.id}') for schema in schema_list]

[('schema uri: ./pipeline/SchemaGen/schema/3', 'schema id:3'),
 ('schema uri: ./pipeline/ImportSchemaGen/schema/4', 'schema id:4')]

Vamos a obtener la primera instancia de `ExampleAnomalies` para obtener la salida de `ExampleValidator`.

In [19]:
# Obtener la primera instancia de ExampleAnomalies
example_anomalies = store.get_artifacts_by_type('ExampleAnomalies')[0]

# Print the artifact id
print(f'Artifact id: {example_anomalies.id}')

Artifact id: 5


Utilizarás el ID del artefacto para obtener los eventos relacionados con él. Vamos a obtener la primera instancia.

In [20]:
# Obtener el primer evento relacionado con el ID
anomalies_id_event = store.get_events_by_artifact_ids([example_anomalies.id])[0]

# Print results
print(anomalies_id_event)

artifact_id: 5
execution_id: 5
path {
  steps {
    key: "anomalies"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1667502242211



Como era de esperar, el tipo de evento será un `OUTPUT` porque es la salida del componente `ExampleValidator`. Como queremos obtener las entradas, podemos rastrearlas a través del id de ejecución.

In [21]:
# Get execution ID
anomalies_execution_id = anomalies_id_event.execution_id

# Get events by the execution ID
events_execution = store.get_events_by_execution_ids([anomalies_execution_id])

# Print results
print(events_execution)

[artifact_id: 2
execution_id: 5
path {
  steps {
    key: "statistics"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1667502241842
, artifact_id: 4
execution_id: 5
path {
  steps {
    key: "schema"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1667502241846
, artifact_id: 5
execution_id: 5
path {
  steps {
    key: "anomalies"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1667502242211
]


Vemos los artefactos que están marcados como `INPUT` arriba representando las estadísticas y las entradas del esquema. Podemos extraer sus IDs mediante programación de la siguiente manera. Verás que obtendrás el ID del artefacto del esquema curado que imprimiste antes.

In [22]:
# Filter INPUT type events
inputs_to_exval = [event.artifact_id for event in events_execution 
                       if event.type == metadata_store_pb2.Event.INPUT]

# Print results
print(inputs_to_exval)

[2, 4]


**Has completado este cuaderno sobre esquemas iterativos y has visto cómo se puede utilizar en un pipeline TFX. También pudiste rastrear el linaje de un artefacto mirando los artefactos, eventos y ejecuciones en el almacén de metadatos. Esto será muy útil en la tarea de esta semana.